In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
model = tf.keras.models.Sequential([ 
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)), 
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'), 
  tf.keras.layers.MaxPooling2D(2,2), 
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
  tf.keras.layers.MaxPooling2D(2,2), 
  tf.keras.layers.Flatten(), 
  tf.keras.layers.Dense(256, activation='relu'), 
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [3]:
model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])

In [4]:
data = tfds.load('horses_or_humans', split='train', as_supervised=True) 
val_data = tfds.load('horses_or_humans', split='test', as_supervised=True)

In [5]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

In [6]:
def augment_img(image, label):
  image, label = normalize_img(image, label)
  image = tf.image.random_flip_left_right(image)
  return image, label

In [7]:
data = data.cache()
data = data.map(augment_img, num_parallel_calls=tf.data.AUTOTUNE)
train_data = data.shuffle(1024).batch(32)
train_data = train_data.prefetch(tf.data.AUTOTUNE)
val_data = val_data.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
val_data = val_data.batch(32)
val_data = val_data.cache()
val_data = val_data.prefetch(tf.data.AUTOTUNE)

In [8]:
% time history = model.fit(train_data, epochs=10, validation_data=val_data, validation_steps=1)

Epoch 1/10
33/33 [==============================] - 17s 93ms/step - loss: 0.8459 - accuracy: 0.7400 - val_loss: 0.3556 - val_accuracy: 0.8750
Epoch 2/10
33/33 [==============================] - 4s 71ms/step - loss: 0.1272 - accuracy: 0.9464 - val_loss: 0.9759 - val_accuracy: 0.8125
Epoch 3/10
33/33 [==============================] - 2s 48ms/step - loss: 0.0214 - accuracy: 0.9942 - val_loss: 1.5251 - val_accuracy: 0.8125
Epoch 4/10
33/33 [==============================] - 2s 47ms/step - loss: 0.0166 - accuracy: 0.9961 - val_loss: 1.7294 - val_accuracy: 0.7188
Epoch 5/10
33/33 [==============================] - 2s 48ms/step - loss: 0.0185 - accuracy: 0.9932 - val_loss: 0.5618 - val_accuracy: 0.8438
Epoch 6/10
33/33 [==============================] - 2s 48ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 1.0553 - val_accuracy: 0.8438
Epoch 7/10
33/33 [==============================] - 2s 48ms/step - loss: 0.0063 - accuracy: 0.9981 - val_loss: 1.0521 - val_accuracy: 0.8125
Epoch 8/10
3

In [9]:
# CPU times: user 37.6 s, sys: 2.73 s, total: 40.3 s
# Wall time: 31.8 s

In [10]:
# CPU times: user 37.5 s, sys: 2.79 s, total: 40.3 s
# Wall time: 43 s

In [11]:
# CPU times: user 1min 41s, sys: 4.19 s, total: 1min 45s
# Wall time: 1min 41s